## Data Merging and Cleaning

This file assembles two "master" dataframes for experimentation. The first dataframe contains every crime throughout each day, with associated daily weather information. The second dataframe contains the number of arrests each day with associated mean weather information.

### How to use:

1. **Download CSV data and place in the ./input directory**
    - [NYPD_Arrest_Data__Year_to_Date_.csv](https://drive.google.com/file/d/1Ee7dSLLK7EdiMwiE94uK3p0SiDd7vTrP/view?usp=sharing)
    - [NYPD_Arrests_Data__Historic_.csv](https://drive.google.com/file/d/1g_Iok1V2NnWKBy0r9qGG7XavUlOhDzWe/view?usp=sharing)
    - [daily_new_york_data.csv](https://drive.google.com/file/d/1_B0xP4ORTzHCG9S4LaEB_yEjC1mvITXU/view?usp=sharing)
2. **Run cells in order from top to bottom**
    - Run all cells in order
    - Run all cells only once
3. **View output in the ./output directory**
    - We can either output files in our program, or call this module directly from another module to receive this data.

In [ ]:
import numpy as np
import pandas as pd

### Arrest data

We can load all arrests, or filter by law codes or by level of offense. If we don't want to filter anything, then everything in the next two cells should be commented out. If we want to filter by one or both, uncomment one or both cells and set the variables to the desired values.

**Dataset**
   - Historic arrest dataset: https://data.cityofnewyork.us/Public-Safety/NYPD-Arrest-Data-Year-to-Date-/uip8-fykc
       - [CSV file from Google Drive](https://drive.google.com/file/d/1_B0xP4ORTzHCG9S4LaEB_yEjC1mvITXU/view?usp=sharing)
   - Current arrest dataset: https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u
       - [CSV file from Google Drive](https://drive.google.com/file/d/1Ee7dSLLK7EdiMwiE94uK3p0SiDd7vTrP/view?usp=sharing)
   - New York State Penal Law Offenses: https://ypdcrime.com/penallawlist.php?tfm_order=DESC&tfm_orderby=code
    
**Resources**
   - New York State Penal Law Offenses: https://ypdcrime.com/penallawlist.php?tfm_order=DESC&tfm_orderby=code
   - Laws of New York: https://ypdcrime.com/penal.law/

In [ ]:
historic_arrests = pd.read_csv('./input/NYPD_Arrests_Data__Historic_.csv')

In [ ]:
current_arrests = pd.read_csv('./input/NYPD_Arrest_Data__Year_to_Date_.csv')

In [ ]:
# Number of dates.
print('Number of unique dates: ' + str(len(current_arrests['ARREST_DATE'].unique()) + len(historic_arrests['ARREST_DATE'].unique())))

In [ ]:
# Number of arrests.
print('Number of arrests: ' + str(len(current_arrests['ARREST_DATE']) + len(historic_arrests['ARREST_DATE'])))

In [ ]:
# Append historic data to current data for a full list.
total_arrests = current_arrests.append(historic_arrests)
total_arrests.columns = total_arrests.columns.str.lower()

In [ ]:
total_arrests.dropna(subset=['pd_desc', 'ofns_desc'], inplace=True)

In [ ]:
def format_date_arrests(date):
    split = date.split('/')
    return split[2] + split[0] + split[1]

In [ ]:
total_arrests['date'] = total_arrests['arrest_date'].apply(format_date_arrests)

In [ ]:
# Only return abbreviated NY penal codes.
def get_law_code_prefix(code):
    if type(code) == str:
        if code[:3] == 'PL ': 
            return code[3:6] # First three numerals.
    return np.nan

In [ ]:
# Create a column with just the first five characters of the law code for filtering.
total_arrests['law_code_abbr'] = total_arrests['law_code'].apply(get_law_code_prefix)

In [ ]:
# Label each arrest with an offense category using the
# law_code_abbr column to identify the offense.

offenses = {
    'Anticipatory Offenses': [100, 115],
    'Offenses Against the Person': [120, 135],
    'Damage to Property Offenses': [140, 150],
    'Theft Offenses': [155, 165],
    'Fraud Offenses': [170, 190],
    'Public Administration Offenses': [195, 215],
    'Public Health, Morals Offenses': [220, 235],
    'Public Order, Privacy': [240, 250],
    'Family, Welfare of Children': [255, 263],
    'Public Safety Offenses': [265, 275]
}

def get_offense(code):
    code = int(code)
    for key, value in offenses.items():
        code_min = value[0]
        code_max = value[1]
        if (code >= code_min) & (code <= code_max):
            return key

In [ ]:
# Ignore arrests that are not categorized.
total_arrests.dropna(subset=['law_code_abbr'], inplace=True)

# Add offense categories to arrests for filtering later.
total_arrests['law_code_desc'] = total_arrests['law_code_abbr'].apply(get_offense)

In [ ]:
# Save total_arrests as a checkpoint for debugging.
arrests = total_arrests

### Weather data

**Dataset**
   - National Weather Service: https://www.weather.gov/okx/CentralParkHistorical
       - [CSV file from Google Drive](https://drive.google.com/file/d/1_B0xP4ORTzHCG9S4LaEB_yEjC1mvITXU/view?usp=sharing)

#### Hourly weather

In [ ]:
hourly_weather = pd.read_csv('./input/daily_new_york_data.csv')

# Grabbing more than temperature just so they're handy if we want to play a bit.
hourly_weather = hourly_weather[['dt', 'temp', 'feels_like', 'temp_min', 'temp_max', 
                                 'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 
                                 'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all']]

In [ ]:
# Sanity check.
print('Total of hours of weather data: ' + str(len(hourly_weather)))

In [ ]:
# Format dates into YYYYMMDD.
from datetime import datetime
hourly_weather['date'] = hourly_weather['dt'].apply(lambda dt: datetime.fromtimestamp(dt).strftime('%Y%m%d'))

#### Daily weather

In [ ]:
# Daily weather averages and extremes.
daily_weather = hourly_weather

# Fill nan with 0.
daily_weather[['rain_1h', 
               'rain_3h', 
               'snow_1h', 
               'snow_3h']] = daily_weather[['rain_1h', 
                                            'rain_3h', 
                                            'snow_1h', 
                                            'snow_3h']].fillna(value=0)

# Aggregate.
daily_weather = daily_weather.groupby(['date']).agg({'temp':'mean', 'feels_like':'mean', 
                                                     'temp_min': 'min', 'temp_max': 'max', 
                                                     'humidity': 'mean', 'wind_speed': 'mean', 
                                                     'wind_deg': 'mean', 'rain_1h': 'mean', 
                                                     'rain_3h': 'mean', 'snow_1h': 'mean', 
                                                     'snow_3h': 'mean', 'clouds_all': 'mean'})

# Date as index.
daily_weather['date'] = daily_weather.index

In [ ]:
# Sanity check.
print('Total number of days: ' + str(len(daily_weather)))

### Final datasets

This dataframe contains more detail, including each arrest's description and location in the city, by borough. This dataset may or may not be used, depending if we have time to do extra fancy visualizations. Otherwise, the next dataframe showing daily summaries are what we'll focus on first.

In [ ]:
# Set date as index for merging.
arrests.set_index(['date'], inplace=True)

#### All arrests

In [ ]:
# Merge arrest data and weather data.
all_arrests = pd.merge(left=arrests, right=daily_weather, how='left',
                        left_index=True, right_index=True)

all_arrests = all_arrests[['pd_desc', 'ofns_desc', 'law_code', 'law_cat_cd',
                           'arrest_boro', 'age_group', 'perp_sex', 'perp_race',
                           'temp', 'feels_like', 'temp_min', 'temp_max', 'humidity', 
                           'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 
                           'snow_1h', 'snow_3h', 'law_code_abbr', 'law_code_desc']]

all_arrests.dropna(inplace=True)

In [ ]:
# Sanity check.
print('Total number of arrests: ' + str(len(all_arrests)))

**Write to file:**

In [ ]:
all_arrests.to_csv('./output/all_arrests.csv')

#### Daily arrests

This dataframe will likely be our primary dataset, since it's giving us a day-by-day arrest count with the mean weather conditions for that day.

In [ ]:
daily_arrests = all_arrests

**Filter arrests by law code and level:**

In [ ]:
# Filter arrests.
daily_arrests = daily_arrests[daily_arrests['law_code_desc'] == 'Offenses Against the Person']
daily_arrests = daily_arrests[daily_arrests['law_cat_cd'] == 'F']

In [ ]:
# Aggregate arrests into daily summaries.
daily_arrests = daily_arrests.groupby(['date']).count()
daily_arrests.drop(columns=['ofns_desc', 'law_code_desc', 'arrest_boro'], inplace=True)
daily_arrests.rename(columns={'pd_desc': 'num_arrests'}, inplace=True)
daily_arrests = daily_arrests[['num_arrests']]
daily_arrests = pd.merge(left=daily_arrests, right=daily_weather, 
                         how='left', left_index=True, right_index=True)

In [ ]:
# Sanity check.
print('Total number of daily arrests: ' + str(daily_arrests['num_arrests'].sum()))

In [ ]:
# Sanity check.
print('Total number of days: ' + str(len(daily_arrests)))

**Write to file:**

In [ ]:
daily_arrests.to_csv('./output/daily_arrests.csv')

### EDA

Display a quick pairplot to quickly see if there's any correleation between number of arrests per day and weather conditions.

In [ ]:
daily_arrests.head()

#### Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(ncols=2, nrows=4, sharey=True, figsize=(12, 24))
ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8 = ax.flatten()
scatter_toggle = False

sns.regplot(x=daily_arrests['temp'], y=daily_arrests['num_arrests'], ax=ax1, scatter=scatter_toggle).set_title("Temp")
sns.regplot(x=daily_arrests['feels_like'], y=daily_arrests['num_arrests'], ax=ax2, scatter=scatter_toggle).set_title("Feels Like")
sns.regplot(x=daily_arrests['temp_min'], y=daily_arrests['num_arrests'], ax=ax3, scatter=scatter_toggle).set_title("Temp: Min")
sns.regplot(x=daily_arrests['temp_max'], y=daily_arrests['num_arrests'], ax=ax4, scatter=scatter_toggle).set_title("Temp: Max")
sns.regplot(x=daily_arrests['rain_1h'], y=daily_arrests['num_arrests'], ax=ax5, scatter=scatter_toggle).set_title("Rain: 1 Hour")
sns.regplot(x=daily_arrests['rain_3h'], y=daily_arrests['num_arrests'], ax=ax6, scatter=scatter_toggle).set_title("Rain: 3 Hours")
sns.regplot(x=daily_arrests['snow_1h'], y=daily_arrests['num_arrests'], ax=ax7, scatter=scatter_toggle).set_title("Snow: 1 Hour")
sns.regplot(x=daily_arrests['snow_3h'], y=daily_arrests['num_arrests'], ax=ax8, scatter=scatter_toggle).set_title("Snow: 3 Hours")

#### Remarks

There's some correlation when we restrict the dataset to particular offense categories and particular offense levels. Heat seems to have a slight positive correlation with number of arrests. Rain and snow have a higher, negative correlations with number of arrests.